In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import collections
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [ ]:
# For inline graphics in Jupyter
%matplotlib inline

# First stage

## Построить диаграмму рассеяния для двух произвольно взятых признаков

In [ ]:
breast_cancer_ds = datasets.load_breast_cancer()

In [ ]:
# We can view target names
# 0 - Malignant
# 1 - Benign
breast_cancer_ds.target_names

In [ ]:
# We can view description of the dataset
print(breast_cancer_ds.DESCR)

In [ ]:
# Lets take 1 and 6 parameters (radius and compactness)
X = breast_cancer_ds.data[:, [0,5]]  # we only take the first two features.
y = breast_cancer_ds.target

# Clear figure
plt.clf()
# Draw scatter plot
# Red ones are malignant, gray are benign
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set1,
            edgecolor='k')

plt.xlabel('Radius (mean)')
plt.ylabel('Compactness (mean)')

plt.show()

## Рассчитать матрицу ковариации исходного набора данных (X) и для исходного набора, спроецированного на главные компоненты (X_reduced).

In [ ]:
X = breast_cancer_ds.data
y = breast_cancer_ds.target

# Normalize data
X = StandardScaler().fit_transform(X)

pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

The covariance is:
maximized - if the two vectors are identical
0 - if they are orthogonal
negative - if they point to opposite directions

In [ ]:
# X covariance matrix
# Need to transpose the matrix, because 
# we need to place observations in columns:

# numpy.cov: each row of m represents a variable, 
#            each column a single observation of all those variables
cov_X = np.cov(X.T)
cov_X

In [ ]:
# X_reduced covariance matrix
cov_X_reduced = np.cov(X_reduced.T)
cov_X_reduced

## Проверить, что главные компоненты ортогональны.

In [ ]:
# Checking by finding cos of an angle between vectors
# Orthoganal = 90 degrees
# Cos90 = 0
cos_angle = np.dot(X_reduced[:, 0], X_reduced[:, 1]) / np.linalg.norm(X_reduced[:, 0]) / np.linalg.norm(X_reduced[:, 1])

if -0.00001 < cos_angle < 0.00001:
    print('Main components are orthogonal')
else:
    print('Main components are NOT orthogonal')

## Сравнить собственные значения матрицы ковариации X со значениями дисперсии главных компонент.

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(cov_X)
eig_vals

They are almost similar

In [ ]:
# print(eig_vals[0])
# print(eig_vals[1])
# print(np.var(X_reduced[:,0]))
# print(np.var(X_reduced[:,1]))

print("Variance/eigenvalue 1 component, in %\n{}".format(
                            np.var(X_reduced[:,0])/eig_vals[0]*100))
print("Variance/eigenvalue 2 component, in %\n{}".format(
                            np.var(X_reduced[:,1])/eig_vals[1]*100))

## Рассчитать total variation (след матрицы ковариации) для X и X_reduced.
##  Показать, что данный параметр не меняется при проецировании на главные компоненты.

In [ ]:
X_trace = np.trace(X)
X_reduced_trace = np.trace(X_reduced)
X_reduced_trace / X_trace

In [ ]:
cov_X_trace = np.trace(cov_X)
cov_X_reduced_trace = np.trace(cov_X_reduced)

print(cov_X_trace)
print(cov_X_reduced_trace)

cov_X_reduced_trace / cov_X_trace

Here we see that 2 main components are showing only 63% of previous data

## Построить графики % объясненной дисперсии: а) для исходных признаков, б) для главных компонент.

In [ ]:
print(round(100 * pca.explained_variance_ratio_[0], 2))
print(round(100 * pca.explained_variance_ratio_[1], 2))
# cov_X_reduced / trace

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(cov_X_reduced)
eig_vals

Graph for our 2 PCA components

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(cov_X_reduced)
tot = sum(eig_vals) # возьмем сумму всех собствееных сзначений

# посчитаем сколько каждый состовляет от общего
var_exp = [(i / tot) * 100 for i in sorted(eig_vals, reverse=True)]

cum_var_exp = np.cumsum(var_exp)

plt.xlabel('component')
plt.ylabel('var, %')

plt.plot(range(0, len(var_exp), 1), var_exp, 'o', range(0, len(var_exp), 1), var_exp, 'k')
plt.plot(range(0, len(cum_var_exp), 1), cum_var_exp, 'o', range(0, len(cum_var_exp), 1), cum_var_exp, 'k')

myax = plt.gca()
for graph_x,graph_y in zip(range(0, len(var_exp), 1), var_exp):
    myax.annotate("(" + str(graph_x) + ", " + str(graph_y) + ")", xy=(graph_x, graph_y + 2))
for graph_x,graph_y in zip(range(0, len(cum_var_exp), 1), cum_var_exp):
    myax.annotate("(" + str(graph_x) + ", " + str(y) + ")", xy=(graph_x, graph_y + 2))

plt.show()

Graph for initial data features

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(cov_X)
tot = sum(eig_vals) # возьмем сумму всех собствееных сзначений

# посчитаем сколько каждый состовляет от общего
var_exp = [(i / tot) * 100 for i in sorted(eig_vals, reverse=True)]

cum_var_exp = np.cumsum(var_exp)

plt.xlabel('component')
plt.ylabel('var, %')

plt.plot(range(0, len(var_exp), 1), var_exp, 'o', range(0, len(var_exp), 1), var_exp, 'k')
plt.plot(range(0, len(cum_var_exp), 1), cum_var_exp, 'o', range(0, len(cum_var_exp), 1), cum_var_exp, 'k')

plt.show()

# Second stage

## Разделить выборку на тренировочную и тестовую (90% и 10%)

In [ ]:
print(str(X.shape))
print(str(y))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

## Обучить классификатор на основе метода k ближайших соседей при фиксированном (произвольном) k.

In [ ]:
# Educate
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)

# Predict
y_pred = neigh.predict(X_test)

# Print report
target_names = ['class 0', 'class 1']
print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test, y_pred))
accuracy_score(y_test, y_pred)

In [ ]:
# Predict
y_pred_train = neigh.predict(X_train)

# Print report
target_names = ['class 0', 'class 1']
print(classification_report(y_train, y_pred_train, target_names=target_names))
print(confusion_matrix(y_train, y_pred_train))
accuracy_score(y_train, y_pred_train)

## Оценить качество классификации на тренировочной/тестовой выборке.

95% - точность на исходных данных

89% - точность на тренировочных данных

## Сделать 10-fold кросс-валидацию при фиксированном k, оценить дисперсию.

In [ ]:
scores = cross_val_score(neigh, X, y, cv=10)
print("Cross-validated scores: \n{}".format(scores))

In [ ]:
print("Variation of cross-validation results = {}".format(np.var(scores))) 

## Построить графики зависимости точности на тренировочном/тестовом наборе от числа k (с дисперсией) для n-fold кросс-валидации для разных значений n (n = 2, 5, 8, 10).

In [ ]:
pred = cross_val_predict(neigh, X, y, cv=10)
accuracy_score(pred, y)

In [ ]:
dct = {}

kf = KFold(n_splits=10, shuffle=True)

for nbr in range(1, 11):
    neigh = KNeighborsClassifier(n_neighbors=nbr)
    for i in range(10):
        if i not in dct:
            dct[nbr] = []
        pred = cross_val_predict(neigh, X, y, cv=kf)
        
        res =  accuracy_score(pred, y)
        
        dct[nbr].append(res)

In [ ]:
dct_mean = {i: np.mean(dct[i]) for i in dct}
dct_mean

In [ ]:
var = np.var(list(dct_mean.values()))
var

In [ ]:
dct_var = {i: np.var(dct[i]) for i in dct}
dct_var

In [ ]:
list_mean = [dct_mean[i] for i in range(1, 11)];

plt.plot(list(dct_mean.keys()), list(dct_mean.values()), 'o', list(dct_mean.keys()), list(dct_mean.values()), 'k')
# plt.plot(list(dct_var.keys()), list(dct_var.values()), 'o', list(dct_var.keys()), list(dct_var.values()), 'k')
# plt.plot(range(0, len(cum_var_exp), 1), cum_var_exp, 'o', range(0, len(cum_var_exp), 1), cum_var_exp, 'k')

plt.show()

## Сделать вывод о сравнительном качестве предсказаний при разных способах оценки точности. Определить наиболее приемлемый диапазон значений k (числа ближайших соседей).

Начиная с k=5, точность уже значительно не повышается

# Stage three

## Осуществить п. 2 не для исходной выборки, а для X_reduced (число главных компонент варьировать, исследовать диапазон k, определенный в п. 2).

## Сравнить качество классификации при обучении классификатора по исходному набору/ по главным компонентам. Исследовать качество классификации при вариации числа главных компонент.

In [ ]:
def test_pca_numbers(components=3):
    pca = PCA(n_components=components)
    X_reduced = pca.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.1, random_state=0)
    
#     neigh = KNeighborsClassifier(n_neighbors=[5, 6, 7, 8, 9, 10])
    neigh = KNeighborsClassifier(n_neighbors=9)
    neigh.fit(X_train, y_train)
#     knn_params = {'knn__n_neighbors': [2, 5, 8, 10]}
#     knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1))])
#     knn_grid = GridSearchCV(knn_pipe, knn_params,
#                         cv=5, n_jobs=-1,
#                         verbose=False)
#     knn_grid.fit(X_test, y_test)
    score = accuracy_score(y_test, neigh.predict(X_test))
    report = {
#         'best_params' : knn_grid.best_params_,
#         'best_score on training set' : knn_grid.best_score_,
        'accuracy score on test set': score,
        'Number of PCs' : components
    }
    
    return report

In [ ]:
comps = [2, 3, 5, 7, 9, 14, 20]

for i in comps:
    print(test_pca_numbers(components=i))

9 соседей, 5 и более главных компонент